# Задание 4

Для каждой функции:

* исследуйте сходимость градиентного спуска с постоянным шагом, сравните полученные результаты для выбранных функций;
* сравните эффективность градиентного спуска с использованием одномерного поиска с точки зрения количества вычислений минимизируемой функции и ее градиентов;
* исследуйте работу методов в зависимости от выбора начальной точки;
* исследуйте влияние нормализации (scaling) на сходимость на примере масштабирования осей плохо обусловленной функции;
* в каждом случае нарисуйте графики с линиями уровня и траекториями методов;